In [1]:
from whispr import *

In [25]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 3
source_plate_type = '384PP_AQ_BP' 

sp_plasmids_file = 'source_plates/221101_arpae_plasmids.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0)
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

filename = '221101_arpae_genex.csv'
layout_genex_file = 'plate_layouts/'+filename 
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = 'mixing_tables/221101_arpae_plasmids.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol='source_plates/221101_arpae_plasmids_updated.xlsx')


Volumes used from each well for this protocol:
{'1_A1': 3.0, '1_P1': 0.3, '1_P2': 0.35, '1_P3': 1.0, '1_P5': 0.3, '1_P6': 0.3, '1_P7': 0.9, '1_P8': 0.4, '1_P9': 0.2}


In [26]:
protocol_genex_df = protocol_genex_df[0]

In [27]:
vol_table_df['Label'].iloc[-2]

'no_pyc'

In [28]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'N24'
for label, dwell in rxn_loc.items():
    for d in dwell:
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv('protocols/'+filename,index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

pyc
K3
pyc
K4
no_pyc
L3
no_pyc
L4
Std
M3
P++
M4
P++
O4
N++
N4
N++
P4
TXTL needed: 20.25


In [37]:
# dilution. everything gets Tris, for 20x -> 57uL
protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
source_plate_type = '6RES_AQ_B' 
tris_well = 'A1'
tris_vol = 57*1000

for dwell in protocol_genex_df['Destination Well'].unique():
    row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
                                'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
    protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [ ]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [42]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = 'source_plates/221024_arpae_buffers.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0)
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = 'mixing_tables/221024_arpae_buffers.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
filename = '221024_arpae_biosyn.csv'
layout_biosyn1_file = 'plate_layouts/'+filename 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol='source_plates/221024_arpae_combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_A1': 13.2, '1_A2': 1.2, '1_B1': 16.5, '1_C1': 8.25, '1_D1': 17.5, '1_D2': 2.0, '1_E1': 40.0, '1_E2': 40.0, '1_E3': 17.5, '1_F1': 3.9, '1_G1': 15.6, '1_H1': 39.0, '1_I1': 5.25, '1_I2': 24.0, '1_J1': 17.5, '1_J2': 2.0, '1_K2': 17.85, '1_L1': 2.85, '1_N1': 12.0, '1_O1': 12.0, '1_P1': 26.85, '2_A2': 745.05, '3_A3': 7.5, '3_B3': 7.5, '3_C3': 7.5, '3_D3': 7.5, '3_E3': 7.5, '3_F3': 15.0, '3_G3': 15.0, '3_H3': 15.0, '3_I3': 15.0, '3_J3': 37.5}


In [43]:
protocol_biosyn_dfs[0].to_csv('protocols/221024_arpae_biosyn_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv('protocols/221024_arpae_biosyn_6RES.csv',index = False)